# Master Layer Setup & Management

This notebook demonstrates how to use the Master Layer Setup script to create and manage the foundational reference tables for your data warehouse.

## Architecture Overview

The **Master Layer** provides the foundational reference data for the entire data warehouse:

- **Clients**: Top-level customer organizations 
- **Practices**: Individual operating practices (main reporting unit)
- **Locations**: Physical/logical locations tied to practices
- **Providers**: Individual providers (dentists, hygienists, etc.)
- **Time Periods**: Canonical time windows for consistent reporting
- **Aliases**: Maps raw identifiers to canonical entities
- **Templates**: Input contract definitions and versioning

## Key Benefits

✅ **Centralized Reference Data**: Single source of truth for all entities  
✅ **Consistent Reporting**: Standardized time periods and entity definitions  
✅ **Flexible Ingestion**: Alias mapping handles messy/changing source data  
✅ **Template Versioning**: Track evolution of client data structures  
✅ **Future-Proof**: Ready for multi-client, multi-practice scaling

In [ ]:
# Import Required Libraries
import sys
import os
from pathlib import Path
import pandas as pd
import sqlalchemy
from datetime import datetime, date
import subprocess

# Add utils to path
sys.path.append('../utils')
from connect_db import get_engine

print("📚 Libraries imported successfully")
print(f"🐍 Python version: {sys.version}")
print(f"🐼 Pandas version: {pd.__version__}")
print(f"🗄️  SQLAlchemy version: {sqlalchemy.__version__}")

# Verify database connection
try:
    engine = get_engine()
    with engine.connect() as conn:
        result = conn.execute("SELECT version()")
        db_version = result.fetchone()[0]
        print(f"✅ Database connected: PostgreSQL")
        print(f"📊 Database version: {db_version.split(',')[0]}")
except Exception as e:
    print(f"❌ Database connection failed: {e}")

In [ ]:
# Import the Master Layer Setup Functions
sys.path.append('.')  # Current directory to import master_layer_setup.py

try:
    from master_layer_setup import (
        create_all_tables, 
        create_single_table, 
        add_client,  # New generic function
        populate_sample_data_for_wso,  # Backward compatibility
        generate_time_periods,
        get_all_table_creation_functions
    )
    print("✅ Master layer setup functions imported successfully")
except ImportError as e:
    print(f"❌ Could not import master layer setup: {e}")
    print("💡 Make sure master_layer_setup.py is in the current directory")

# Alternative: Use subprocess to call the script
def run_master_script(action, **kwargs):
    """Run the master layer setup script via subprocess"""
    cmd = ["python", "master_layer_setup.py", "--action", action]
    
    for key, value in kwargs.items():
        cmd.extend([f"--{key.replace('_', '-')}", str(value)])
    
    try:
        result = subprocess.run(cmd, capture_output=True, text=True, check=True)
        print("✅ Command executed successfully:")
        print(result.stdout)
        if result.stderr:
            print("⚠️  Warnings/Info:")
            print(result.stderr)
        return True
    except subprocess.CalledProcessError as e:
        print(f"❌ Command failed with exit code {e.returncode}:")
        print(f"STDOUT: {e.stdout}")
        print(f"STDERR: {e.stderr}")
        return False
    except Exception as e:
        print(f"❌ Unexpected error: {e}")
        return False

print("🔧 Setup complete - ready to manage master layer!")

## Step 1: Create Master Layer Tables

The first step is creating all the master layer tables. This establishes the foundational schema for reference data.

### Tables to be created:
1. **`master.clients`** - Top-level customer organizations
2. **`master.practices`** - Individual operating practices  
3. **`master.locations`** - Physical/logical locations
4. **`master.providers`** - Individual providers (dentists, etc.)
5. **`master.time_periods`** - Canonical time windows
6. **`master.practice_aliases`** - Raw practice identifier mappings
7. **`master.provider_aliases`** - Raw provider identifier mappings
8. **`master.input_templates`** - Client input contracts
9. **`master.input_template_versions`** - Template version history
10. **`master.input_template_fields`** - Field-level template definitions

In [ ]:
# Option 1: Create all tables at once (recommended for new setup)
print("🚀 Creating all master layer tables...")

try:
    create_all_tables()
    print("\n🎉 All master tables created successfully!")
except Exception as e:
    print(f"❌ Error creating tables: {e}")
    print("\n💡 Trying alternative approach with subprocess...")
    success = run_master_script("create_all")
    if success:
        print("✅ Tables created via subprocess")

In [ ]:
# Option 2: Create individual tables (for selective updates)
# Uncomment and run any of these to create specific tables:

# create_single_table('clients')
# create_single_table('practices') 
# create_single_table('providers')
# create_single_table('time_periods')

# List all available tables
table_functions = get_all_table_creation_functions()
print("📋 Available master tables:")
for i, table_name in enumerate(table_functions.keys(), 1):
    print(f"   {i:2d}. master.{table_name}")

print(f"\n💡 To create a single table: create_single_table('table_name')")
print(f"💡 To create via script: run_master_script('create_table', table='table_name')")

## Step 2: Populate Sample Data for WSO

After creating the tables, we need to populate them with foundational data for Wall Street Orthodontics (WSO).

This includes:
- **Client record**: WSO as a client organization
- **Practice record**: Main WSO practice 
- **Provider records**: Dr. Shapiro and general practice providers
- **Time periods**: Standard monthly/quarterly/yearly periods
- **Aliases**: Mapping raw identifiers from source data

In [ ]:
# Populate sample master data for Wall Street Orthodontics
print("📝 Populating sample data for WSO...")

try:
    populate_sample_data_for_wso()
    print("\n✅ WSO sample data populated!")
except Exception as e:
    print(f"❌ Error populating data: {e}")
    print("\n💡 Trying via subprocess...")
    success = run_master_script("populate_sample", client="wso")
    if success:
        print("✅ Sample data populated via subprocess")

## Step 2a: Add Any New Client (Interactive & Scalable)

The new `add_client()` function is designed to be scalable and work with any client. It supports both interactive and programmatic approaches.

In [ ]:
# Option 1: Fully Interactive - prompts for all client information
# Uncomment to try:
# new_client_id = add_client()

# Option 2: Semi-Interactive - provide some data, prompt for missing
sample_client_data = {
    'client_name': 'ABC Dental Group',
    'client_tag': 'abc_dental',
    'primary_contact_name': 'Dr. Sarah Johnson'
    # Missing fields will be prompted for interactively
}

print("🔧 Example: Adding a client with partial data (interactive mode)")
print("Data provided:", sample_client_data)
print("💡 In interactive mode, you would be prompted for missing information")

# To actually run this, uncomment:
# new_client_id = add_client(sample_client_data, interactive=True)

In [ ]:
# Option 3: Non-Interactive - provide complete data programmatically
complete_client_data = {
    'client_name': 'Modern Orthodontics LLC',
    'client_tag': 'modern_ortho',
    'billing_entity': 'Modern Orthodontics LLC',
    'primary_contact_name': 'Dr. Michael Chen',
    'primary_contact_email': 'mchen@modernortho.com',
    'primary_contact_phone': '(555) 123-4567',
    'practices': [{
        'practice_name': 'Modern Orthodontics - Downtown',
        'practice_code': 'MOD-DT',
        'practice_type': 'orthodontist',
        'city': 'San Francisco',
        'state': 'CA',
        'email': 'info@modernortho.com',
        'phone': '(555) 123-4567'
    }]
}

print("🤖 Example: Adding a complete client programmatically (non-interactive)")
print("This approach is perfect for bulk imports or API integrations")
print()

# To actually run this, uncomment:
# client_id = add_client(complete_client_data, interactive=False)
# print(f"Client created with ID: {client_id}")

In [ ]:
# Generate standard time periods (2024-2027)
print("📅 Generating standard time periods...")

try:
    generate_time_periods(start_year=2024, end_year=2027)
    print("\n✅ Time periods generated!")
except Exception as e:
    print(f"❌ Error generating time periods: {e}")
    print("\n💡 Trying via subprocess...")
    success = run_master_script("generate_time_periods", start_year=2024, end_year=2027)
    if success:
        print("✅ Time periods generated via subprocess")

## Step 3: Validation and Testing

Verify that the master layer setup was successful and the data is properly structured.

In [ ]:
# Validate master layer setup
def validate_master_layer():
    """Check that all master tables exist and have expected data"""
    engine = get_engine()
    
    print("🔍 MASTER LAYER VALIDATION")
    print("=" * 50)
    
    expected_tables = [
        'clients', 'practices', 'locations', 'providers', 
        'time_periods', 'practice_aliases', 'provider_aliases',
        'input_templates', 'input_template_versions', 'input_template_fields'
    ]
    
    validation_results = {}
    
    with engine.connect() as conn:
        # Check if master schema exists
        schema_check = conn.execute("""
            SELECT EXISTS(SELECT 1 FROM information_schema.schemata WHERE schema_name = 'master')
        """).fetchone()[0]
        
        print(f"📊 Master schema exists: {'✅' if schema_check else '❌'}")
        
        if not schema_check:
            print("❌ Master schema not found - run table creation first!")
            return False
        
        # Check each table
        for table in expected_tables:
            try:
                # Check table exists
                table_exists = conn.execute(f"""
                    SELECT EXISTS(SELECT 1 FROM information_schema.tables 
                    WHERE table_schema = 'master' AND table_name = '{table}')
                """).fetchone()[0]
                
                if table_exists:
                    # Get row count
                    count_result = conn.execute(f"SELECT COUNT(*) FROM master.{table}").fetchone()
                    row_count = count_result[0]
                    validation_results[table] = {'exists': True, 'rows': row_count}
                    print(f"✅ master.{table}: {row_count} rows")
                else:
                    validation_results[table] = {'exists': False, 'rows': 0}
                    print(f"❌ master.{table}: Table not found")
                    
            except Exception as e:
                validation_results[table] = {'exists': False, 'error': str(e)}
                print(f"❌ master.{table}: Error - {e}")
    
    # Summary
    existing_tables = sum(1 for r in validation_results.values() if r.get('exists'))
    total_rows = sum(r.get('rows', 0) for r in validation_results.values())
    
    print(f"\n📈 SUMMARY")
    print(f"   Tables created: {existing_tables}/{len(expected_tables)}")
    print(f"   Total rows: {total_rows}")
    
    if existing_tables == len(expected_tables):
        print(f"\n🎉 Master layer validation PASSED!")
        return True
    else:
        print(f"\n⚠️  Master layer validation FAILED - missing tables")
        return False

# Run validation
validation_success = validate_master_layer()

In [ ]:
# Query sample data to verify setup
def show_sample_data():
    """Display sample data from key master tables"""
    engine = get_engine()
    
    print("📋 SAMPLE MASTER DATA")
    print("=" * 50)
    
    queries = {
        "Clients": "SELECT client_id, client_name, client_tag, client_status FROM master.clients",
        "Practices": "SELECT practice_id, client_id, practice_name, practice_type, city, state FROM master.practices",
        "Providers": "SELECT provider_id, practice_id, provider_first_name, provider_last_name, provider_type FROM master.providers",
        "Time Periods (Recent)": """
            SELECT period_type, period_name, period_start_date, period_end_date, is_complete 
            FROM master.time_periods 
            WHERE period_year >= 2024 
            ORDER BY period_start_date DESC 
            LIMIT 5
        """
    }
    
    for title, query in queries.items():
        try:
            print(f"\n📊 {title}:")
            df = pd.read_sql(query, engine)
            if not df.empty:
                print(df.to_string(index=False))
            else:
                print("   (No data)")
        except Exception as e:
            print(f"   ❌ Error: {e}")

# Show sample data if validation passed
if validation_success:
    show_sample_data()
else:
    print("⚠️  Skipping sample data display - validation failed")

## Usage Examples

### Command Line Usage

The `master_layer_setup.py` script supports multiple actions:

```bash
# Create all tables
python master_layer_setup.py --action create_all

# Create a specific table
python master_layer_setup.py --action create_table --table clients

# Add a new client interactively (prompts for all info)
python master_layer_setup.py --action add_client

# Add a new client with some predefined data
python master_layer_setup.py --action add_client \
  --client-name "Smile Center" \
  --client-tag "smile_center" \
  --contact-name "Dr. Jennifer Smith" \
  --contact-email "jsmith@smilecenter.com"

# Add client non-interactively (no prompts - requires all data)
python master_layer_setup.py --action add_client \
  --client-name "Quick Dental" \
  --client-tag "quick_dental" \
  --contact-name "Dr. Bob Wilson" \
  --contact-email "bwilson@quickdental.com" \
  --non-interactive

# Generate time periods
python master_layer_setup.py --action generate_time_periods --start-year 2024 --end-year 2027

# Populate WSO sample data (backward compatibility)
python master_layer_setup.py --action populate_sample --client wso
```

### Python Usage

You can also import and use the functions directly:

```python
from master_layer_setup import create_all_tables, add_client

# Create tables
create_all_tables()

# Add client interactively
client_id = add_client()

# Add client programmatically
client_data = {
    'client_name': 'Elite Dental Care',
    'client_tag': 'elite_dental',
    'primary_contact_name': 'Dr. Lisa Park',
    'primary_contact_email': 'lpark@elitedental.com'
}
client_id = add_client(client_data, interactive=False)
```

### Key Improvements

✅ **Scalable**: Works with any client, not just hardcoded WSO  
✅ **Interactive**: Prompts for missing information when needed  
✅ **Flexible**: Supports both manual and programmatic usage  
✅ **Complete Setup**: Handles clients, practices, and providers in one flow  
✅ **Smart Defaults**: Suggests codes and types based on input  
✅ **Conflict Handling**: Updates existing records if client_tag already exists  

## Next Steps

1. ✅ **Master layer is ready** - foundational reference data is in place
2. 🔄 **Bronze layer ingestion** - can now proceed with any client's data ingestion  
3. 🏗️ **Silver layer development** - build standardized business logic layer
4. ✨ **Gold layer reports** - create production-ready outputs

The master layer now provides a truly scalable foundation for your entire data warehouse!